In [2]:
import torch

/Users/petersong/Library/Caches/pypoetry/virtualenvs/transformer-8abVYlk8-py3.11/lib/python3.11/site-packages/torch/nn/modules/transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


In [3]:
#Simplified self-attention 코드
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [4]:
#x2에 해당하는 'journey'의 어텐션 스코어 연산
query = inputs[1]
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [5]:
#이전에 계산한 어텐션 점수를 정규화하여 어텐션 가중치를 계산
atten_weight_2_tmp = attn_scores_2 / attn_scores_2.sum()
print("어텐션 가중치: ", atten_weight_2_tmp)
print("합계: ", atten_weight_2_tmp.sum())

어텐션 가중치:  tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
합계:  tensor(1.0000)


In [6]:
#softmax를 활용한 어텐션 정규화
#해당 함수 사용시 큰 값이나 작은 값을 처리할 때 발생 할 수 있는 수치적 불안정이 발생 할 수 있음
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weight_2_naive = softmax_naive(attn_scores_2)
print("어텐션 가중치: ", attn_weight_2_naive)
print("합계: ", attn_weight_2_naive.sum())

어텐션 가중치:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
합계:  tensor(1.)


In [7]:
#파이토치 소프트맥스 사용
#dim을 0을 선택함에 따라 (6,) 형태의 텐서에 대해 소프트맥스 함수를 적용
attn_weight_2 = torch.softmax(attn_scores_2, dim=0)
print("어텐션 가중치: ", attn_weight_2)
print("합계: ", attn_weight_2.sum())

어텐션 가중치:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
합계:  tensor(1.)


In [8]:
#z(2)의 어텐션 스코어 계산
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    print(attn_weight_2[i])
    context_vec_2 += attn_weight_2[i] * x_i
print(context_vec_2)

tensor(0.1385)
tensor(0.2379)
tensor(0.2333)
tensor(0.1240)
tensor(0.1082)
tensor(0.1581)
tensor([0.4419, 0.6515, 0.5683])


In [9]:
#모든 입력 토큰에 대한 어텐션 weights 계산
attn_scores = torch.empty(6,6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [10]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [11]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("두 번째 행의 합계:", row_2_sum)
print("모든 행의 합계:", attn_weights.sum(dim=-1))


두 번째 행의 합계: 1.0
모든 행의 합계: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [12]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [13]:
print("이전의 두 번째 문맥 벡터:", context_vec_2)

이전의 두 번째 문맥 벡터: tensor([0.4419, 0.6515, 0.5683])


In [14]:
#Sacled dot-product attention
x_2 = inputs[1]
#input 임베딩 사이즈
d_in = inputs.shape[1]
#output 임베딩 사이즈
d_out = 2

In [15]:
torch.manual_seed(123)
#가중치 업데이트를 위해서는 True로 해야함. 여기서는 간단한 구현을 위해 False로 함
#기울기를 연산하지 않는 경우는 모델의 일부를 프리징하거나 기울기가 필요하지 않을 경우 사용됨
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [16]:
#weight parameters와 attention weights은 동일하지 않음
#attention weights는 주어진 문백 벡터가 입력의 다른 부분에 얼마나 의존하는지를 나타내는 동적인 값임
query_2 = x_2 @ W_query 
key_2 = x_2 @ W_key 
value_2 = x_2 @ W_value
print(query_2)
print(key_2)
print(value_2)

tensor([0.4306, 1.4551])
tensor([0.4433, 1.1419])
tensor([0.3951, 1.0037])


In [17]:
keys = inputs @ W_key 
values = inputs @ W_value
print("키의 크기:", keys.shape)
print("값의 크기:", values.shape)

키의 크기: torch.Size([6, 2])
값의 크기: torch.Size([6, 2])


In [18]:
keys

tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]])

In [19]:
values

tensor([[0.1855, 0.8812],
        [0.3951, 1.0037],
        [0.3879, 0.9831],
        [0.2393, 0.5493],
        [0.1492, 0.3346],
        [0.3221, 0.7863]])

In [20]:
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


In [21]:
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [22]:
keys

tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]])

In [23]:
query_2

tensor([0.4306, 1.4551])

In [24]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


In [25]:
#z(2) 계산 결과
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


트랜스포머에서 쿼리, 키, 벨류란?

**쿼리(Query)**

쿼리는 현재 모델이 집중하고 있는 항목을 나타냅니다. 예를 들어, 문장 "The dog chased the cat across the street"에서 모델이 현재 "dog"이라는 단어를 처리하고 있다면, "dog"이 쿼리가 됩니다. 쿼리는 모델이 다른 입력 시퀀스의 부분에 얼마나 집중해야 하는지를 결정하는 데 사용됩니다.

**키(Key)**

키는 데이터베이스의 키와 유사하게 사용됩니다. 각 입력 시퀀스의 항목(예를 들어, 문장의 각 단어)은 키를 가지고 있습니다. 이러한 키는 쿼리와 일치하는 데 사용됩니다. 예를 들어, "The dog chased the cat across the street"에서 각 단어("The", "dog", "chased", ...)는 키를 가지고 있습니다.

**벨류(Value)**

벨류는 키-벨류 쌍의 벨류와 유사하게 사용됩니다. 벨류는 입력 항목의 실제 내용 또는 표현을 나타냅니다. 모델이 쿼리와 가장 관련 있는 키(입력의 부분)를 결정하면, 해당하는 벨류를 가져옵니다. 예를 들어, "dog"이라는 쿼리에 대해 모델이 "chased"라는 키와 가장 관련이 있다고 결정하면, "chased"라는 벨류를 가져옵니다.

이러한 쿼리, 키, 벨류의 개념은 데이터베이스의 정보 검색과 유사하게 사용됩니다. 모델은 쿼리를 사용하여 입력 시퀀스의 다른 부분에 얼마나 집중해야 하는지를 결정하고, 키를 사용하여 쿼리와 일치하는 부분을 찾고, 벨류를 사용하여 해당하는 내용을 가져옵니다.

예를 들어, 다음의 문장에서 모델이 "it"이라는 단어를 처리하고 있다면:

"The animal didn't cross the street because it was too tired."

* 쿼리: "it"
* 키: "The", "animal", "didn't", "cross", "the", "street", "because", "it", "was", "too", "tired"
* 벨류: 각 키에 해당하는 내용(예를 들어, "animal"의 벨류는 "동물"과 같은 의미를 가짐)

모델은 쿼리 "it"과 키를 비교하여 가장 관련 있는 키를 찾고, 해당하는 벨류를 가져옵니다. 이 경우, 모델은 "it"과 "animal"이 가장 관련이 있다고 결정하고, "animal"의 벨류를 가져옵니다.

In [26]:
import torch.nn as nn
class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        valeus = x @ self.W_value
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        context_vec = attn_weights @ valeus
        return context_vec

In [27]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [28]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        context_vec = attn_weights @ values
        return context_vec


In [29]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))


tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


In [30]:
queries = sa_v2.W_query(inputs)     #1
keys = sa_v2.W_key(inputs) 
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)
#1 이전 사용한 SelfAttention_v2 객체의 쿼리와 키 가중치 행렬을 재사용


tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [31]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [32]:
masked_simple = attn_weights * mask_simple
print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


In [33]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
# maksed_simple_norm = masked_simple / row_sums
row_sums2 = masked_simple.sum(dim=-1, keepdim=False)


In [34]:
print(row_sums)

tensor([[0.1921],
        [0.3700],
        [0.5357],
        [0.6775],
        [0.8415],
        [1.0000]], grad_fn=<SumBackward1>)


In [35]:
print(row_sums2)

tensor([0.1921, 0.3700, 0.5357, 0.6775, 0.8415, 1.0000],
       grad_fn=<SumBackward1>)


In [36]:
maksed_simple_norm = masked_simple / row_sums
print(maksed_simple_norm)


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


In [37]:
#torch.triu 함수는 상삼각행렬(upper triangular matrix)을 반환
#diagonal=1을 지정하면, 대각선 위의 요소만 1로 유지하고 나머지 요소는 0으로 설정됨
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
print(mask)

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])


In [38]:
#masked_fill 함수는 mask 텐서의 True인 위치에 -torch.inf 값을 채움
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


In [39]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [40]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)    #1
example = torch.ones(6, 6)      #2
print(dropout(example))
#1 드롭아웃 비율을 50%로 설정합니다.
#2 여기서 우리는 1로 이루어진 행렬을 만듭니다.

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


In [41]:
example = torch.ones(6, 6)
print(example)

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])


In [42]:
print(dropout(example))

tensor([[2., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 0., 2.],
        [0., 0., 2., 2., 0., 2.],
        [0., 2., 0., 2., 2., 0.],
        [0., 2., 2., 2., 2., 2.],
        [2., 2., 0., 0., 2., 2.]])


In [43]:
#seed를 고정하여 드롭아웃 결과가 일관되게 유지할  수 있음. 단 OS에 따라 결과가 다를 수 있음
#https://github.com/pytorch/pytorch/issues/121595
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.8966, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4921, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4350, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


In [44]:
#2개의 인풋을 중접해서 멀티 헤드 어텐션 구현을 위해 사용
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


In [45]:
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask',
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2)
        #질문 할 것
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens],-torch.inf)
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        attn_weights = self.dropout(attn_weights)

        context_vec = attn_weights @ values
        return context_vec

In [46]:
torch.manual_seed(123)
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)
context_vecs = ca(batch)
print("context_vecs.shape:", context_vecs.shape)


context_vecs.shape: torch.Size([2, 6, 2])


In [47]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(
                 d_in, d_out, context_length, dropout, qkv_bias
             ) 
             for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)


In [48]:
torch.manual_seed(123)
context_length = batch.shape[1] # 입력에 포함된 토큰의 수
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)


tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


In [57]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads    #1
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)    #2
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)         #3
        queries = self.W_query(x)    #3
        values = self.W_value(x)     #3

        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)       #4
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)  
        queries = queries.view(                                             
            b, num_tokens, self.num_heads, self.head_dim                    
        )                                                                   

        keys = keys.transpose(1, 2)          #5
        queries = queries.transpose(1, 2)    #5
        values = values.transpose(1, 2)      #5

        attn_scores = queries @ keys.transpose(2, 3)   #6
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]    #7

        attn_scores.masked_fill_(mask_bool, -torch.inf)     #8

        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2)   #9
        context_vec = context_vec.contiguous().view(
            b, num_tokens, self.d_out
        )
        context_vec = self.out_proj(context_vec)    #11
        return context_vec


In [58]:
torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


In [63]:
sample = torch.rand(2,6,4)
print(sample)

tensor([[[0.3443, 0.6800, 0.9998, 0.2855],
         [0.9753, 0.2518, 0.7204, 0.6959],
         [0.6397, 0.8954, 0.2979, 0.6314],
         [0.5028, 0.1239, 0.3786, 0.1661],
         [0.7211, 0.5449, 0.5490, 0.3483],
         [0.5024, 0.3445, 0.6437, 0.9856]],

        [[0.5757, 0.2785, 0.1946, 0.5382],
         [0.1291, 0.1242, 0.1746, 0.3302],
         [0.5370, 0.8443, 0.6937, 0.8831],
         [0.1861, 0.5422, 0.0556, 0.7868],
         [0.6042, 0.9836, 0.1444, 0.9010],
         [0.9221, 0.9043, 0.5713, 0.9546]]])


In [68]:
test_transpose_q = sample
test_transpose_k = sample

In [69]:
test_transpose_q_view = test_transpose_q.view(2, 6, 2, 2)
print(test_transpose_q_view)

tensor([[[[0.3443, 0.6800],
          [0.9998, 0.2855]],

         [[0.9753, 0.2518],
          [0.7204, 0.6959]],

         [[0.6397, 0.8954],
          [0.2979, 0.6314]],

         [[0.5028, 0.1239],
          [0.3786, 0.1661]],

         [[0.7211, 0.5449],
          [0.5490, 0.3483]],

         [[0.5024, 0.3445],
          [0.6437, 0.9856]]],


        [[[0.5757, 0.2785],
          [0.1946, 0.5382]],

         [[0.1291, 0.1242],
          [0.1746, 0.3302]],

         [[0.5370, 0.8443],
          [0.6937, 0.8831]],

         [[0.1861, 0.5422],
          [0.0556, 0.7868]],

         [[0.6042, 0.9836],
          [0.1444, 0.9010]],

         [[0.9221, 0.9043],
          [0.5713, 0.9546]]]])


In [70]:
test_transpose_k_view = test_transpose_k.view(2, 6, 2, 2)
print(test_transpose_k_view)

tensor([[[[0.3443, 0.6800],
          [0.9998, 0.2855]],

         [[0.9753, 0.2518],
          [0.7204, 0.6959]],

         [[0.6397, 0.8954],
          [0.2979, 0.6314]],

         [[0.5028, 0.1239],
          [0.3786, 0.1661]],

         [[0.7211, 0.5449],
          [0.5490, 0.3483]],

         [[0.5024, 0.3445],
          [0.6437, 0.9856]]],


        [[[0.5757, 0.2785],
          [0.1946, 0.5382]],

         [[0.1291, 0.1242],
          [0.1746, 0.3302]],

         [[0.5370, 0.8443],
          [0.6937, 0.8831]],

         [[0.1861, 0.5422],
          [0.0556, 0.7868]],

         [[0.6042, 0.9836],
          [0.1444, 0.9010]],

         [[0.9221, 0.9043],
          [0.5713, 0.9546]]]])


In [74]:
test_transpose_k_t = test_transpose_k.transpose(1,2)
print(test_transpose_k_t)

tensor([[[0.3443, 0.9753, 0.6397, 0.5028, 0.7211, 0.5024],
         [0.6800, 0.2518, 0.8954, 0.1239, 0.5449, 0.3445],
         [0.9998, 0.7204, 0.2979, 0.3786, 0.5490, 0.6437],
         [0.2855, 0.6959, 0.6314, 0.1661, 0.3483, 0.9856]],

        [[0.5757, 0.1291, 0.5370, 0.1861, 0.6042, 0.9221],
         [0.2785, 0.1242, 0.8443, 0.5422, 0.9836, 0.9043],
         [0.1946, 0.1746, 0.6937, 0.0556, 0.1444, 0.5713],
         [0.5382, 0.3302, 0.8831, 0.7868, 0.9010, 0.9546]]])


In [75]:
test_transpose_k_t_t = test_transpose_k.transpose(2,3)
print(test_transpose_k_t_t)

IndexError: Dimension out of range (expected to be in range of [-3, 2], but got 3)